# Inisialisasi dan Pengaturan Lingkungan

Sel ini berfungsi sebagai fondasi dari keseluruhan notebook, memastikan semua dependensi dan konfigurasi siap sebelum proses inti dijalankan.

In [ ]:
# Impor Global, Pengaturan Path, Konfigurasi Logger, Impor Fungsi Fitur

# Impor Global yang Mungkin Diperlukan di Seluruh Notebook
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import logging # Impor logging standar
from typing import Optional, Callable, Any, Union, List, Tuple # Pastikan semua type hints yang akan digunakan ada di sini

# --- 1. Pengaturan Path ---
project_root_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))


if project_root_path not in sys.path:
    sys.path.append(project_root_path)
    # Pesan ini hanya akan muncul sekali jika path baru ditambahkan
    print(f"Path proyek ditambahkan: {project_root_path}")


# --- 2. Impor dan Konfigurasi Logger ---
try:
    from src.utils.logger import setup_logging, get_logger
    # Panggil setup_logging SEKALI SAJA per sesi kernel notebook.
    if not logging.getLogger().hasHandlers(): # Cek jika root logger belum punya handler
         setup_logging(log_level=logging.INFO) # Atur level sesuai kebutuhan (misal: INFO atau DEBUG)
    logger = get_logger(__name__) # Mendapatkan logger untuk notebook ini
    logger.info("Logger kustom berhasil di-setup dan didapatkan untuk notebook.")
except ImportError:
    # Fallback ke basic config jika modul logger kustom tidak ditemukan
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__) # Mendapatkan logger default
    logger.warning("Menggunakan basicConfig untuk logger karena src.utils.logger tidak ditemukan atau error impor.")
except Exception as e:
    # Fallback jika ada error lain saat setup logger kustom
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)
    logger.error(f"Error saat setup logger kustom: {e}. Menggunakan basicConfig.", exc_info=True)


# --- 3. Impor Fungsi Ekstraksi Fitur dari Modul ---
try:
    from src.features.feature_extractor_shortener import extract_features # Ini adalah nama fungsi dari file
    logger.info("Fungsi 'extract_features' dari feature_extractor.py berhasil diimpor.")
except ImportError:
    logger.critical("GAGAL mengimpor 'extract_features' dari src.features.feature_extractor.")
    logger.critical("Pastikan sys.path sudah benar, file ada, dan tidak ada error syntax di feature_extractor.py.")
    logger.critical("Inferensi TIDAK AKAN BEKERJA DENGAN BENAR tanpa fungsi ekstraksi fitur yang valid.")
    def extract_features(url_string: str) -> Optional[list]:
        logger.error("!!! MENGGUNAKAN FUNGSI 'extract_features' PLACEHOLDER !!! Hasil prediksi akan SALAH.")

        num_expected_features = 21 
        logger.warning(f"Placeholder mengembalikan {num_expected_features} fitur acak.")
        return list(np.random.rand(num_expected_features)) if url_string else None
except Exception as e:
    logger.critical(f"Error tak terduga saat mengimpor 'extract_features': {e}", exc_info=True)
    def extract_features(url_string: str) -> Optional[list]:
        logger.error(f"!!! MENGGUNAKAN FUNGSI 'extract_features' PLACEHOLDER karena error impor: {e} !!! Hasil prediksi akan SALAH.")
        num_expected_features = 21 
        logger.warning(f"Placeholder mengembalikan {num_expected_features} fitur acak.")
        return list(np.random.rand(num_expected_features)) if url_string else None


logger.info("Setup awal tahap 1 notebook untuk inferensi URL tunggal selesai.") # Pesan log yang lebih spesifik

# --- 4. Konfigurasi Path untuk Model dan Preprocessor ---
# Path untuk Model
MODEL_DIR_RELATIVE_TO_SRC = Path("models_trained")
MODEL_NAME = "best_recall_model_url shortener case.h5"

MODEL_DIR_ABSOLUTE = Path(project_root_path) / MODEL_DIR_RELATIVE_TO_SRC # Model ada di dalam src
SAVED_MODEL_PATH = MODEL_DIR_ABSOLUTE / MODEL_NAME


logger.info(f"Path model yang akan digunakan: {SAVED_MODEL_PATH}")
logger.info("Setup awal tahap 2 notebook (path) untuk inferensi URL tunggal selesai.")

Path proyek ditambahkan: d:\Capstone\ML - Phishing Detection
2025-06-12 19:48:26 - root - INFO - Konfigurasi logging selesai.
2025-06-12 19:48:26 - __main__ - INFO - Logger kustom berhasil di-setup dan didapatkan untuk notebook.
2025-06-12 19:48:26 - __main__ - INFO - Fungsi 'extract_features' dari feature_extractor.py berhasil diimpor.
2025-06-12 19:48:26 - __main__ - INFO - Setup awal tahap 1 notebook untuk inferensi URL tunggal selesai.
2025-06-12 19:48:26 - __main__ - INFO - Path model yang akan digunakan: d:\Capstone\ML - Phishing Detection\models_trained\best_recall_model_url shortener case.h5
2025-06-12 19:48:26 - __main__ - INFO - Setup awal tahap 2 notebook (path) untuk inferensi URL tunggal selesai.


# Pemuatan Model Machine Learning
Sel ini berfokus pada satu tugas krusial: memuat model Keras yang telah dilatih dari file.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Memuat Model ---
logger.info(f"Mencoba memuat model dari: {SAVED_MODEL_PATH}")
loaded_model = None # Inisialisasi sebagai None
try:
    if SAVED_MODEL_PATH.exists(): # Pastikan file model ada
        loaded_model = keras.models.load_model(SAVED_MODEL_PATH)
        logger.info("Model berhasil dimuat.")
        loaded_model.summary() # Tampilkan ringkasan model untuk verifikasi
    else:
        logger.error(f"File model tidak ditemukan di: {SAVED_MODEL_PATH}")
except Exception as e:
    logger.error(f"Gagal memuat model dari {SAVED_MODEL_PATH}: {e}", exc_info=True)


2025-06-12 19:48:30 - __main__ - INFO - Mencoba memuat model dari: d:\Capstone\ML - Phishing Detection\models_trained\best_recall_model_url shortener case.h5
2025-06-12 19:48:30 - absl - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-06-12 19:48:30 - __main__ - INFO - Model berhasil dimuat.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,139 (43.52 KB)

 Trainable params: 11,137 (43.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

# Definisi Fungsi Prediksi Inti
Sel ini mengenkapsulasi seluruh logika prediksi ke dalam satu fungsi yang bersih dan dapat digunakan kembali


In [ ]:
# Fungsi untuk Inferensi URL Tunggal

def predict_url_type(url_string: str, model: keras.Model,
                     # 'extract_features' akan digunakan langsung dari impor global
                     preprocessor_obj: Optional[Any] = None,
                     threshold: float = 0.5) -> Tuple[Optional[str], Optional[float]]:
    """
    Menerima URL, mengekstrak fitur, melakukan pra-pemrosesan (jika ada), membuat prediksi,
    dan mengembalikan tipe prediksi beserta probabilitasnya.

    Args:
        url_string (str): URL yang akan diprediksi.
        model (keras.Model): Model Keras yang sudah dilatih dan dimuat.
        preprocessor_obj (Optional[Any]): Objek preprocessor (misal, scaler) yang sudah di-fit.
                                       Jika None, tidak ada pra-pemrosesan scaler yang diterapkan.
        threshold (float): Ambang batas untuk klasifikasi jika output model adalah sigmoid.

    Returns:
        Tuple[Optional[str], Optional[float]]: (tipe_prediksi, probabilitas_phishing)
                                                atau (None, None) jika error.
    """
    if not url_string:
        logger.warning("Input URL kosong.")
        return None, None

    logger.info(f"Memproses URL: {url_string}")

    # 1. Ekstraksi Fitur menggunakan fungsi yang sudah diimpor dari feature_extractor.py
    try:
        list_of_features = extract_features(url_string) # Memanggil fungsi yang diimpor dari Sel 1
        if list_of_features is None:
            logger.error(f"Ekstraksi fitur mengembalikan None untuk URL: {url_string}")
            return None, None
        
        # Konversi ke NumPy array float32
        features_1d = np.array(list_of_features, dtype=np.float32)
        logger.debug(f"Fitur (1D) diekstrak: {features_1d.shape} -> {features_1d[:5]}...")

    except Exception as e:
        logger.error(f"Error saat ekstraksi fitur untuk URL '{url_string}': {e}", exc_info=True)
        return None, None
    
    # Validasi jumlah fitur terhadap input model (
    if model.input_shape[1] is not None and features_1d.shape[0] != model.input_shape[1]:
        logger.error(f"Jumlah fitur yang diekstrak ({features_1d.shape[0]}) tidak sesuai dengan input yang diharapkan model ({model.input_shape[1]}).")
        return None, None

    # 2. Pra-pemrosesan Fitur (jika ada preprocessor)
    # Fitur perlu diubah menjadi 2D array (1 sampel, N fitur)
    if features_1d.ndim == 1:
        features_2d_for_model = features_1d.reshape(1, -1)
    else: 
        logger.warning(f"Fitur yang diekstrak sudah {features_1d.ndim}D. Diharapkan 1D sebelum reshape.")
        features_2d_for_model = features_1d # Asumsikan sudah (1, N_fitur) jika bukan 1D

    if preprocessor_obj:
        try:
            logger.debug(f"Menerapkan preprocessor pada fitur shape: {features_2d_for_model.shape}")
            features_processed = preprocessor_obj.transform(features_2d_for_model) # transform() biasanya mengharapkan 2D
            logger.debug("Preprocessor berhasil diterapkan.")
        except Exception as e:
            logger.error(f"Error saat menerapkan preprocessor: {e}", exc_info=True)
            return None, None
    else:
        features_processed = features_2d_for_model 

    logger.debug(f"Fitur setelah pra-pemrosesan (siap untuk model): {features_processed.shape} -> {features_processed[0, :5]}...")

    # 3. Membuat Prediksi
    try:
        prediction_proba = model.predict(features_processed) # Input ke model harus 2D
        logger.debug(f"Probabilitas mentah dari model: {prediction_proba}")

        # Interpretasi output model berdasarkan arsitektur output layer
        num_output_neurons = model.layers[-1].units
        activation_output = model.layers[-1].activation.__name__ # Mendapatkan nama aktivasi

        if num_output_neurons == 1 and activation_output == 'sigmoid':
            prob_phishing = prediction_proba[0, 0]
            predicted_label_int = 1 if prob_phishing > threshold else 0
        elif num_output_neurons > 1 and activation_output == 'softmax': 
            predicted_label_int = np.argmax(prediction_proba, axis=1)[0]
            prob_phishing = prediction_proba[0, 1] 
        else:
            logger.error(f"Struktur output layer model tidak dikenali (neurons: {num_output_neurons}, activation: {activation_output}). Tidak bisa interpretasi prediksi.")
            return None, None
        
        # TODO: 
        label_mapping = {0: 'Aman', 1: 'Phishing'} 
        predicted_type = label_mapping.get(predicted_label_int, f"Label Int Tidak Diketahui: {predicted_label_int}")
        
        logger.info(f"Prediksi untuk '{url_string}': Tipe='{predicted_type}', Prob_Phishing={prob_phishing:.4f} (Threshold: {threshold if num_output_neurons == 1 else 'argmax'})")
        return predicted_type, prob_phishing

    except Exception as e:
        logger.error(f"Error saat membuat prediksi untuk URL '{url_string}': {e}", exc_info=True)
        return None, None

In [ ]:
# Loop Interaktif untuk Input Pengguna

if 'loaded_model' in locals() and loaded_model: # Pastikan model sudah dimuat
    print("\n--- Mode Inferensi URL Tunggal ---")
    print("Ketik 'exit' atau 'quit' untuk keluar.")
    while True:
        user_url = input("\nMasukkan URL yang ingin Anda periksa: ").strip()
        if user_url.lower() in ['exit', 'quit']:
            logger.info("Keluar dari mode inferensi.")
            break
        if not user_url:
            print("URL tidak boleh kosong. Silakan coba lagi.")
            continue
        
        # TODO: 
        prediction_threshold = 0.5 

        predicted_type, probability = predict_url_type(
            user_url,
            loaded_model, 
            threshold=prediction_threshold 
        )

        if predicted_type is not None:
            print(f"--> Hasil Prediksi untuk '{user_url}':")
            print(f"    Tipe Terdeteksi: {predicted_type}")
            if probability is not None:
                print(f"    Probabilitas (Phishing): {probability:.4f}")
        else:
            print(f"--> Tidak dapat membuat prediksi untuk '{user_url}'. Periksa log untuk detail.")
else:
    print("Model belum dimuat (variabel 'loaded_model' tidak ditemukan atau None). Jalankan Sel 2 terlebih dahulu.")
    logger.error("Inferensi tidak bisa dijalankan karena model belum dimuat.")


--- Mode Inferensi URL Tunggal ---
Ketik 'exit' atau 'quit' untuk keluar.
2025-06-12 19:49:27 - __main__ - INFO - Memproses URL: https://www.vouchers.news/alfamart?gp=1&js=1&uuid=1749732292.0025708484&other_args=eyJ1cmkiOiAiL2FsZmFtYXJ0IiwgImFyZ3MiOiAiIiwgInJlZmVyZXIiOiAiIiwgImFjY2VwdCI6ICJ0ZXh0L2h0bWwsYXBwbGljYXRpb24veGh0bWwreG1sLGFwcGxpY2F0aW9uL3htbDtxPTAuOSxpbWFnZS9hdmlmLGltYWdlL3dlYnAsaW1hZ2UvYXBuZywqLyo7cT0wLjgsYXBwbGljYXRpb24vc2lnbmVkLWV4Y2hhbmdlO3Y9YjM7cT0wLjcifQ==
2025-06-12 19:49:27 - FeatureExtractor - INFO - Memulai ekstraksi fitur untuk URL: https://www.vouchers.news/alfamart?gp=1&js=1&uuid=1749732292.0025708484&other_args=eyJ1cmkiOiAiL2FsZmFtYXJ0IiwgImFyZ3MiOiAiIiwgInJlZmVyZXIiOiAiIiwgImFjY2VwdCI6ICJ0ZXh0L2h0bWwsYXBwbGljYXRpb24veGh0bWwreG1sLGFwcGxpY2F0aW9uL3htbDtxPTAuOSxpbWFnZS9hdmlmLGltYWdlL3dlYnAsaW1hZ2UvYXBuZywqLyo7cT0wLjgsYXBwbGljYXRpb24vc2lnbmVkLWV4Y2hhbmdlO3Y9YjM7cT0wLjcifQ==
2025-06-12 19:49:27 - FeatureExtractor - INFO - Mengecek apakah https://www.vouchers.news/a